# 앙상블 학습

정형 데이터 (Structured Data) 기준으로는 가장 뛰어난 상과를 내는 알고리즘 -> 앙상블 학습

대부분의 앙상블 학습 -> 트리 기반

## Random Forest

1. 결정트리를 랜덤하게 만들어 트리의 숲을 만듦

2. 각 결정트리의 예측을 종합해 최종 예측을 만듦

- 과대적합(overfitting)에 안전!

### 데이터 분할

데이터가 1000개면, 각 트리마다 1000개 데이터를 복원추출함. 이때 중복을 허용 (우연히 같은데이터만 1000개 가능)

노드 분할시, 분류 / 회귀의 특성 선택방식이 다름(분류: 개수를 루트함, 회귀: 특성을 다 씀)

기본값 100개의 트리를 만들어서

- 분류: 다수결 투표

- 회귀: 100개의 평균


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

warnings.filterwarnings('ignore')

# data import
df_wine = pd.read_csv('./wine.csv')

df_wine.head()

# ttsplit
X = df_wine[['alcohol', 'sugar', 'pH']].to_numpy()
y = df_wine['class'].to_numpy()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [2]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(rf, X_train, y_train, return_train_score=True, n_jobs=-1)
print('CV_train score:', np.mean(scores['train_score']))
print('CV_test score :', np.mean(scores['test_score']))

rf.fit(X_train, y_train)
# rf: 트리 100개를 돌려서 나온 종합 결과 - 특정 특성에 과도하게 몰리지 않음 -> 단일 트리만 사용했을 때보다 과대적합이 줄어드는 결과
print(rf.feature_importances_)  # 변수별 중요도, 어떤 변수가 많이 사용되었는지

CV_train score: 0.9973541965122431
CV_test score : 0.8905151032797809
[0.23167441 0.50039841 0.26792718]


In [3]:
rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)
# oob(Out Of Bag) 샘플: 트리 100개를 만들 때, 안 쓰인(훈련에 참여 안한) 데이터가 있을 수 있다. - 남은 데이터들을 모아서 만든 샘플
# oob 샘플로 점수를 확인하면 검증샘플과 같은 역할을 한다

rf.fit(X_train, y_train)
print(rf.score(X_train, y_train), rf.oob_score_)

0.996921300750433 0.8934000384837406


In [6]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform, randint

params = {
    # DT의 HP
    'max_depth': randint(3, 50),
    'min_samples_split': randint(2, 20),
    'min_samples_leaf': randint(1, 20),
    # RF의 HP
    'max_features': ['sqrt', 'log2', None],  # 노드별 특성 추출 기준
    'n_estimators': randint(50, 300),  # 트리 개수
}

rscv = RandomizedSearchCV(
    rf,
    params, 
    n_jobs=-1,
    n_iter=100,
    random_state=42,
)

rscv.fit(X_train, y_train)

,estimator,RandomForestC...ndom_state=42)
,param_distributions,"{'max_depth': <scipy.stats....001EBFEB7AB10>, 'max_features': ['sqrt', 'log2', ...], 'min_samples_leaf': <scipy.stats....001EBFFF7CEF0>, 'min_samples_split': <scipy.stats....001EBFFF5E650>, ...}"
,n_iter,100
,scoring,None
,n_jobs,-1
,refit,True
,cv,None
,verbose,0
,pre_dispatch,'2*n_jobs'
,random_state,42
,error_score,nan


In [9]:
print('최적 param', rscv.best_params_)
print('최고 교차검증 점수', rscv.best_score_)
print('Test score:' ,rscv.score(X_test, y_test))
print('특성 중요도:', rscv.best_estimator_.feature_importances_)

최적 param {'max_depth': 14, 'max_features': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 217}
최고 교차검증 점수 0.8878202043384912
Test score: 0.8807692307692307
특성 중요도: [0.22248576 0.53167438 0.24583986]
